In [3]:
import sympy as sp
import numpy as np

from SymPT import * 

from IPython.display import display, Math, Latex, Markdown

# Defining H

In [4]:
# ------------- Defining the symbols for the Hamiltonian ----------------
hbar = sp.symbols('hbar', positive=True, real=True)
omega_x = sp.symbols('omega_x', positive=True, real=True) # Frequency of x confinement
lx = sp.Symbol('l_x', positive=True, real=True)         # Length of x confinement
m = sp.Symbol('m', positive=True, real=True)                    # Effective mass of the particle on Si

ky = RDSymbol('k_y', positive=True, real=True)  
kB = RDSymbol('k_B', positive=True, real=True)  
B = sp.symbols('B', real=True)          # Total Magnetic field perpendicular to the plane of xy confinement B = B0 + b0, where b0 is genereted by the skyrmion
omega_y = omega_x / ky                  # Frequency of y confinement in terms of omega_x
omega_c = - omega_x / kB                # Frequency of omega_c = eB/m in terms of omega_x


# Effective frequencies affter diagonalization of H0 using the minimal coupling substitution
kp = sp.symbols('k_+', positive=True, real=True)
km = sp.symbols('k_-', positive=True, real=True)
# Omega_3_2 = sp.sqrt((omega_x**2 - omega_y**2)**2 + 2 * omega_c**2 * (omega_x**2 + omega_y**2) + omega_c**4)
omega_p = omega_x / kp              #1/sp.sqrt(2) * sp.sqrt(omega_x**2 + omega_y**2 + omega_c**2 + Omega_3_2)
omega_m = omega_x / km              #1/sp.sqrt(2) * sp.sqrt(omega_x**2 + omega_y**2 + omega_c**2 - Omega_3_2)

# -----------------------------------------------------------------------------
# Auxilliary variables
Kp = sp.symbols('K_+', positive=True, real=True)
Km = sp.symbols('K_-', positive=True, real=True)

subs_Kp = {
    Kp : 2 * kp / (kp**2 + 1),
    Km : 2 * km / (km**2 + 1)
}


In [5]:
# ------------- Defining the position momentum basis for the Hamiltonian ----------------
# Defining the bosonic operators of the new basis after diagonalization of H0
# a_+ and a_- are the bosonic operators of the new basis

a_1 = BosonOp('a_+')
ad_1 = Dagger(a_1)
a_2 = BosonOp('a_-')
ad_2 = Dagger(a_2)

# -----------------------------------------------------------------------------
# Defining the position operators x and y in the new basis after diagonalization of H0 and using the bosonic operators of the new basis
theta = sp.Symbol('theta', real=True)               # Parameter used on the diagonalization of H0
beta = - hbar / lx**2 * sp.sqrt(1 / (Kp*Km*ky))     # Parameter used on the diagonalization of H0
alpha = 1 / beta

c1_Omega1 = Kp / omega_x
c2_Omega2 = Kp * ky / omega_x

# Substitutions for the lenghts of the confinement
subs_lenght = {
    sp.sqrt(hbar / (m * omega_x)): lx,
    sp.sqrt( m * omega_x / hbar): 1 / lx,
    sp.sqrt( m * omega_x * hbar): hbar / lx,
}

# Position Operators
x = Operator('x')
y = Operator('y')
px = Operator('p_x')
py = Operator('p_y')

# Position operators in the new basis
x_T = (sp.sqrt(hbar/(2*m) * c1_Omega1) * sp.cos(theta) * (ad_1 + a_1) - sp.I * sp.sqrt(hbar*m * 1 / (2 * c2_Omega2)) * alpha * sp.sin(theta) * (ad_2 - a_2)).subs(subs_lenght)
y_T = (sp.sqrt(hbar/(2*m) * c2_Omega2) * sp.cos(theta) * (ad_2 + a_2) - sp.I * sp.sqrt(hbar*m * 1 / (2 * c1_Omega1)) * alpha * sp.sin(theta) * (ad_1 - a_1)).subs(subs_lenght)
px_T = (sp.sqrt(hbar/(2*m)* c2_Omega2) * beta * sp.sin(theta) * (ad_2 + a_2) + sp.I * sp.sqrt(hbar * m * 1 / (2 * c1_Omega1)) * sp.cos(theta) * (ad_1 - a_1)).subs(subs_lenght)
py_T = (sp.sqrt(hbar/(2*m)* c1_Omega1) * beta * sp.sin(theta) * (ad_1 + a_1) + sp.I * sp.sqrt(hbar * m * 1 / (2 * c2_Omega2)) * sp.cos(theta) * (ad_2 - a_2)).subs(subs_lenght)

transformation = {
    x : x_T,
    y : y_T,
    px : px_T,
    py : py_T
}

a, b = sp.symbols('a b', real=True, positive=True)
subs_theta = {
    theta : sp.Rational(1,2) * sp.atan(-a/b)
}

subs_theta.update({
    sp.cos(theta) * sp.sin(theta) : (sp.cos(theta) * sp.sin(theta)).subs(subs_theta).trigsimp().simplify(),
    sp.sin(theta)**2 : (sp.sin(theta)**2).subs(subs_theta).trigsimp().simplify(),
    sp.cos(theta)**2 : (sp.cos(theta)**2).subs(subs_theta).trigsimp().simplify(),
})

subs_ab = {
    sp.sqrt(a**2 + b**2) : (1/kp**2 - 1/km**2).factor(),
    a : 2 / kB * sp.sqrt(1 / (Kp*Km*ky)),
    b : (ky**2 - 1) / ky**2,
}

display_dict(transformation)

# -----------------------------------------------------------------------------
# ------------- Defining the spin basis for the Hamiltonian ----------------

Spin = RDBasis('sigma', 2)
s0, sx, sy, sz = Spin.basis # Pauli operators

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Lab Frame

In [6]:
kz = RDSymbol('k_z', positive=True, real=True)
Omega_z = omega_x / kz

n1 = RDSymbol('n_1', order=1, positive=True, integer=True)          # b1=n1/lx
n2 = RDSymbol('n_2', order=2, positive=True, integer=True)          # b2=n2/lx**2

H0 = ((sp.factor_terms((hbar * omega_p * ad_1 * a_1 + hbar * omega_m * ad_2 * a_2).expand()).expand() - sp.Rational(1,2) * hbar * Omega_z * sz) / (hbar * omega_x)).expand()
V1 = (sp.factor_terms(sp.Rational(1,2) * hbar * Omega_z * n1 / lx * 1/B * (x_T * sx + y_T * sy)) / (hbar * omega_x)).expand()
V2 = (sp.factor_terms(sp.Rational(1,2) * hbar * Omega_z * n2 / lx**2 * 1/B * (x_T**2  + y_T**2) * sz) / (hbar * omega_x)).expand()

H = H0 + V1 + V2

In [7]:
Eff_frame = EffectiveFrame(H, subspaces=[Spin])
Eff_frame.solve(max_order=2, full_diagonalization=True)
result = Eff_frame.get_H('dict_matrix')

The EffectiveFrame object has been initialized successfully.

Effective Frame

╭────────┬─────────┬─────────────╮
│  Name  │  Type   │  Dimension  │
├────────┼─────────┼─────────────┤
│ sigma  │ Finite  │     2x2     │
├────────┼─────────┼─────────────┤
│  a_+   │ Bosonic │      ∞      │
├────────┼─────────┼─────────────┤
│  a_-   │ Bosonic │      ∞      │
╰────────┴─────────┴─────────────╯

Effective Hamiltonian: 	Not computed yet. To do so, run `solve` method. 




Solving for each order: 100%|██████████| 2/2 [00:00<00:00,  2.08it/s]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Converting to dictionary (matrix) form: 100%|██████████| 6/6 [00:00<00:00, 76.07it/s]


In [8]:
qubit_freq = (result[1][1] - result[1][0]).expand().subs(subs_theta).trigsimp()

In [9]:
sp.factor_terms((qubit_freq - 1/(2*kz)).collect(sp.sqrt(a**2+b**2)).subs(sp.sqrt(a**2+b**2), (1/kp**2 - 1/km**2).factor()).subs(a, 2 / kB * sp.sqrt(1 / (Kp*Km*ky))).collect(b).subs(subs_ab).subs(kp*km,ky).expand().factor().collect(Kp).collect(Km)).subs(kp*km,ky).subs(subs_Kp).expand().factor().subs(kp*km,ky).collect([kp,km]).collect(kB*ky**3 + 2*kB*ky**2 + kB*ky - 2*ky**2*kz).collect(kB*ky**4*kz + kB*ky**3*kz + 2*kB*ky**2*kz+kB*ky*kz+kB*kz).collect(kB*ky**2*kz+kB*kz).subs(kB*ky**5 + 2*kB*ky**4 + 2*kB*ky**3 - 2*ky**4*kz, ky**2*(kB*ky**3 + 2*kB*ky**2 + kB*ky - 2*ky**2*kz)+kB*ky**3).collect(kB*ky**3 + 2*kB*ky**2+ kB*ky -2*ky**2*kz)

(k_B*k_y**5*n_1**2 - 2*k_B*k_y**5*n_2*B - 2*k_B*k_y**4*k_z**2*n_2*B + 2*k_B*k_y**4*n_1**2 - 4*k_B*k_y**4*n_2*B - 4*k_B*k_y**3*k_z**2*n_2*B - 8*k_B*k_y**3*k_z*B**2 + 2*k_B*k_y**3*n_1**2 - 4*k_B*k_y**3*n_2*B - 4*k_B*k_y**2*k_z**2*n_2*B - 8*k_B*k_y**2*k_z*B**2 + 2*k_B*k_y**2*n_1**2 - 4*k_B*k_y**2*n_2*B - 4*k_B*k_y*k_z**2*n_2*B - 8*k_B*k_y*k_z*B**2 + k_B*k_y*n_1**2 - 2*k_B*k_y*n_2*B - 2*k_B*k_z**2*n_2*B - 4*k_B*k_z*B**2*k_+**4 - 4*k_B*k_z*B**2*k_-**4 - 2*k_y**4*k_z*n_1**2 - 2*k_y**2*k_z*n_1**2 + k_+**3*(k_B*k_y**2*k_z*n_1**2 - 2*k_B*k_y**2*k_z*n_2*B - 4*k_B*k_y*k_z*n_2*B - 4*k_B*k_y*B**2 - 4*k_B*k_z**2*B**2 + k_B*k_z*n_1**2 - 2*k_B*k_z*n_2*B) + k_+**2*(k_B*k_y**3*n_1**2 - 2*k_B*k_y**3*n_2*B - 2*k_B*k_y**2*k_z**2*n_2*B - 4*k_B*k_y**2*k_z*B**2 + 2*k_B*k_y**2*n_1**2 - 4*k_B*k_y**2*n_2*B - 4*k_B*k_y*k_z**2*n_2*B - 8*k_B*k_y*k_z*B**2 + k_B*k_y*n_1**2 - 2*k_B*k_y*n_2*B - 2*k_B*k_z**2*n_2*B - 4*k_B*k_z*B**2 - 2*k_y**2*k_z*n_1**2) + k_+*(k_B*k_y**4*k_z*n_1**2 - 2*k_B*k_y**4*k_z*n_2*B + k_B*k_y**3*

In [10]:
kE = sp.Symbol('k_E', positive=True, real=True)
omega_D = sp.Symbol('omega_D', positive=True, real=True)
t = sp.Symbol('t', positive=True, real=True)

E0 = - hbar * omega_x / (2 * lx) * 1/ kE**(3/2) * sp.cos(omega_D * t)
H_drive = E0 * x_T / (hbar * omega_x)   # Drive Hamiltonian in units of hbar*omega_x
H_drive

-cos(omega_D*t)*(sqrt(2)*sqrt(K_+)*l_x*cos(theta)*(Dagger(a_+) + a_+)/2 + sqrt(2)*I*sqrt(K_-)*l_x*sin(theta)*(Dagger(a_-) - a_-)/2)/(2*k_E**1.5*l_x)

In [11]:
E0

-hbar*omega_x*cos(omega_D*t)/(2*k_E**1.5*l_x)

In [12]:
Eff_drive = Eff_frame.rotate(H_drive, return_form='operator')

Projecting to operator form: 100%|██████████| 13/13 [01:49<00:00,  8.45s/it]


In [13]:
Eff_drive = Eff_frame.rotate(H_drive, return_form='operator')

Projecting to operator form: 100%|██████████| 13/13 [01:37<00:00,  7.53s/it]


In [14]:
k3 = sp.Symbol('k_3', positive=True, real=True)
result = group_by_operators(Eff_drive)[sx].factor().collect(sp.sqrt(Kp*Km*ky)*sp.cos(theta)*sp.sin(theta)).collect(Kp*sp.cos(theta)**2).collect(Km*sp.sin(theta)**2).subs(subs_theta).expand().factor().collect(sp.sqrt(a**2+b**2)).subs(sp.sqrt(a**2+b**2), (1/kp**2 - 1/km**2).factor()).subs(a, 2 / kB * sp.sqrt(1 / (Kp*Km*ky))).collect(b).subs(subs_ab).subs(kp*km,ky).expand().factor().collect(Kp).collect(Km).subs(subs_Kp).expand().factor().subs(kp*km,ky).collect([kp,km]).collect(kB*ky**2 - kB*kz**2 - ky**2*kz).subs(((kB*ky**2 - kB*kz**2 - ky**2*kz)*ky**2).expand(), (kB*ky**2 - kB*kz**2 - ky**2*kz)*ky**2).collect(kB*ky**2 - kB*kz**2 - ky**2*kz).subs((kp**2 + 1)*(km**2+1),((kp**2 + 1)*(km**2+1)).expand().subs(kp*km,ky))
result

-k_z*n_1*(k_B*k_y**2 - k_B*k_z**2 - k_y**2*k_z)*cos(omega_D*t)/(4*k_B*B*k_E**1.5*(-k_z + k_+)*(-k_z + k_-)*(k_z + k_+)*(k_z + k_-))

# Co-moving Frame

In [15]:
H0_expr = px**2/(2*m) + py**2/(2*m) + m/2 * (omega_x**2 + sp.Rational(1,4) * omega_c**2) * x**2 + m/2 * (omega_y**2 + sp.Rational(1,4) * omega_c**2) * y**2  + sp.Rational(1,2) * omega_c * (px * y - py * x)
H0_sym = Operator('H_0')
display_dict({
    H0_sym : H0_expr
})

display(Markdown('### Transformed Hamiltonian in the co-moving frame'))

# Co-moving Frame T = exp(i/hbar d(t) px)
# T H0 T^dag + ihbar dot(T) T^dag, TxT^dag = x - d(t)
d_sym = sp.Symbol('d', real=True)
ddt_sym = sp.Symbol('d_t', real=True)
H_transformed_dict = group_by_operators(H0_expr.subs(x, x - d_sym).expand().subs(H0_expr.expand(), H0_sym) - ddt_sym * px)
display_dict(H_transformed_dict)

<IPython.core.display.Math object>

### Transformed Hamiltonian in the co-moving frame

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
# Rotating the Unperturbed Hamiltonian to the co-moving frame
H_drive = E0 * x

# Solving for d(t) in the transformed Hamiltonian such that the drive Hamiltonian is in the co-moving frame
# i.e. The driving Hamiltonian is eliminated from the transformed Hamiltonian
d = sp.nsimplify(sp.solve((H_transformed_dict[x]*x + H_drive ), d_sym)[0]).subs(subs_lenght)
d

-2*k_B**2*l_x*cos(omega_D*t)/(k_E**(3/2)*(4*k_B**2 + 1))

In [17]:
# Rotating the perturbation to the co-moving frame
V1_sym = Operator('V_1')
V2_sym = Operator('V_2')

V1_expr = sp.factor_terms(sp.Rational(1,2) * hbar * Omega_z * n1 / lx * 1/B * (x * sx + y * sy)) 
V1_transformed_dict = group_by_operators(V1_expr.subs(x, x - d_sym).expand().subs(V1_expr.expand(), V1_sym))
V2_expr = sp.factor_terms(sp.Rational(1,2) * hbar * Omega_z * n2 / lx**2 * 1/B * (x**2 + y**2) * sz)
V2_transformed_dict = group_by_operators(V2_expr.subs(x, x - d_sym).expand().subs(V2_expr.expand(), V2_sym))

display(Markdown('### Transformed Perturbation Hamiltonians in the co-moving frame'))
display(Markdown('#### First Order perturbation $V_1$'))
display_dict(V1_transformed_dict)
display(Markdown('#### First Order perturbation $V_2$'))
display_dict(V2_transformed_dict)

### Transformed Perturbation Hamiltonians in the co-moving frame

#### First Order perturbation $V_1$

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### First Order perturbation $V_2$

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [18]:
H_co_moving_drive = ((H_transformed_dict[px] * px_T + H_transformed_dict[py] * py_T ) / (hbar * omega_x)).expand()
display(H_co_moving_drive)

sqrt(2)*d_t*sin(theta)*Dagger(a_-)/(2*sqrt(K_-)*l_x*omega_x) + sqrt(2)*d_t*sin(theta)*a_-/(2*sqrt(K_-)*l_x*omega_x) - sqrt(2)*I*d_t*cos(theta)*Dagger(a_+)/(2*sqrt(K_+)*l_x*omega_x) + sqrt(2)*I*d_t*cos(theta)*a_+/(2*sqrt(K_+)*l_x*omega_x) + sqrt(2)*d*sin(theta)*Dagger(a_+)/(4*k_B*sqrt(k_y)*sqrt(K_-)*l_x) + sqrt(2)*d*sin(theta)*a_+/(4*k_B*sqrt(k_y)*sqrt(K_-)*l_x) - sqrt(2)*I*d*cos(theta)*Dagger(a_-)/(4*k_B*sqrt(k_y)*sqrt(K_+)*l_x) + sqrt(2)*I*d*cos(theta)*a_-/(4*k_B*sqrt(k_y)*sqrt(K_+)*l_x)

In [19]:
Eff_co_moving_drive = Eff_frame.rotate(H_co_moving_drive, return_form='operator')

Projecting to operator form:   0%|          | 0/13 [23:53<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
kp_2_val = sp.nsimplify(1/ (1/2 * (1 + 1/ky**2 + 1/kB**2 + 1/k3**2))).factor()
km_2_val = sp.nsimplify(1/ (1/2 * (1 + 1/ky**2 + 1/kB**2 - 1/k3**2))).factor()

In [ ]:
expr1 = (1/kp_2_val + 1/km_2_val -1).factor()
expr2 = (1/kp**2 + 1/km**2-1).factor().subs(kp*km,ky)

display(expr1)
display(expr2)

subs_sol = {
    expr2.as_numer_denom()[0] : sp.solve(expr1 - expr2, expr2.as_numer_denom()[0])[0].factor().subs(kp*km,ky)
}

display_dict(subs_sol)

Eff_Drive_sy = group_by_operators(Eff_co_moving_drive)[sy].factor().collect(sp.sqrt(Kp*Km*ky)*sp.cos(theta)*sp.sin(theta)).collect(Kp*sp.cos(theta)**2).collect(Km*sp.sin(theta)**2).subs(subs_theta).expand().factor().collect(sp.sqrt(a**2+b**2)).subs(sp.sqrt(a**2+b**2), (1/kp**2 - 1/km**2).factor()).subs(a, 2 / kB * sp.sqrt(1 / (Kp*Km*ky))).collect(b).subs(subs_ab).subs(kp*km,ky).expand().factor().collect(Kp).collect(Km).subs(subs_Kp).expand().factor().subs(kp*km,ky).collect(kB**2).collect(kz**2*ky).subs(subs_sol).factor().collect(kB*ky**2*kz).collect(kB)
display(Eff_Drive_sy)

(k_B**2 + k_y**2)/(k_B**2*k_y**2)

(-k_y**2 + k_+**2 + k_-**2)/k_y**2

<IPython.core.display.Math object>

-n_1*d_t*(k_B**2*(2*k_y**2 - 2*k_z**2) + k_B*k_y**2*k_z*(-k_z**2 - 1) - k_y**2*k_z**2)/(4*k_B**2*B*l_x*omega_x*(-k_z + k_+)*(-k_z + k_-)*(k_z + k_+)*(k_z + k_-))

In [ ]:
expr1 = (1/kp_2_val + 1/km_2_val +1).factor()
expr2 = (1/kp**2 + 1/km**2+1).factor().subs(kp*km,ky)

display(expr1)
display(expr2)

subs_sol = {
    expr2.as_numer_denom()[0] : sp.solve(expr1 - expr2, expr2.as_numer_denom()[0])[0].factor().subs(kp*km,ky)
}

display_dict(subs_sol)

Eff_Drive_sx = group_by_operators(Eff_co_moving_drive)[sx].factor().collect(sp.sqrt(Kp*Km*ky)*sp.cos(theta)*sp.sin(theta)).collect(Kp*sp.cos(theta)**2).collect(Km*sp.sin(theta)**2).subs(subs_theta).expand().factor().collect(sp.sqrt(a**2+b**2)).subs(sp.sqrt(a**2+b**2), (1/kp**2 - 1/km**2).factor()).subs(a, 2 / kB * sp.sqrt(1 / (Kp*Km*ky))).collect(b).subs(subs_ab).subs(kp*km,ky).expand().factor().collect(Kp).collect(Km).subs(subs_Kp).expand().factor().subs(kp*km,ky).expand().factor().collect(kB*kz**2).subs(subs_sol).factor().collect(kB**2*ky**2).collect(kB*kz) + V1_transformed_dict[sx] / (hbar * omega_x)
display(Eff_Drive_sx)

(2*k_B**2*k_y**2 + k_B**2 + k_y**2)/(k_B**2*k_y**2)

(k_y**2 + k_+**2 + k_-**2)/k_y**2

<IPython.core.display.Math object>

-n_1*d/(2*k_z*B*l_x) - n_1*d*(k_B**2*k_y**2*(2*k_z**2 - 2) + k_B*k_z*(k_y**2 + k_z**2) + k_y**2*k_z**2)/(8*k_B**3*B*l_x*(-k_z + k_+)*(-k_z + k_-)*(k_z + k_+)*(k_z + k_-))

In [ ]:
H_Eff_Drive = Eff_Drive_sy * sy + Eff_Drive_sx * sx
H_Eff_Drive.subs(d_sym, d).subs(ddt_sym, sp.diff(d, t))

-n_1*omega_D*(k_B**2*(2*k_y**2 - 2*k_z**2) + k_B*k_y**2*k_z*(-k_z**2 - 1) - k_y**2*k_z**2)*sin(omega_D*t)*sigma_2/(2*B*k_E**(3/2)*omega_x*(4*k_B**2 + 1)*(-k_z + k_+)*(-k_z + k_-)*(k_z + k_+)*(k_z + k_-)) + (k_B**2*n_1*cos(omega_D*t)/(k_z*B*k_E**(3/2)*(4*k_B**2 + 1)) + n_1*(k_B**2*k_y**2*(2*k_z**2 - 2) + k_B*k_z*(k_y**2 + k_z**2) + k_y**2*k_z**2)*cos(omega_D*t)/(4*k_B*B*k_E**(3/2)*(4*k_B**2 + 1)*(-k_z + k_+)*(-k_z + k_-)*(k_z + k_+)*(k_z + k_-)))*sigma_1